In [2]:
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

In [3]:
# Year and Month
date = ['2020', '1']

In [4]:
def send_request(date):
    '''API request'''
    
    base_url = 'https://api.nytimes.com/svc/archive/v1'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + '3YIxSGJ8fF20rV8LAKKKPx05mNoB9AFl'
    response = requests.get(url).json()
    time.sleep(6)
    return response

In [5]:
def parse_response(response):
    '''API parsing and turn into DataFrame'''
    
    data = {
        'date': [],
        'url' : [],
        'headline': [],  
        'articles' : [],
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []
        }
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        data['date'].append(date)
        data['headline'].append(article['headline']['main']) 
        data['url'].append(article['web_url'])
        data['articles'].append(article['snippet'])
        if 'section' in article:
            data['section'].append(article['section_name'])
        else:
            data['section'].append(None)
        data['doc_type'].append(article['document_type'])
        if 'type_of_material' in article: 
            data['material_type'].append(article['type_of_material'])
        else:
            data['material_type'].append(None)
        keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
        data['keywords'].append(keywords)
    return pd.DataFrame(data) 

In [6]:
def extract_label(x):
    '''extract lebels from url'''
    
    df[x] = df['url']
    df[x] = df[x].str.replace(r'(https?:\/\/www.nytimes.com\/(interactive)\/\d+\/\d+\/\d+\/)','', regex=True)
    df[x] = df[x].str.replace(r'(https?:\/\/www.nytimes.com\/\d+\/\d+\/\d+\/)','', regex=True)
    df[x] = df[x].str.replace(r'(https?:\/\/www.nytimes.com\/(slideshow)\/\d+\/\d+\/\d+\/)','', regex=True)
    df[x] = df[x].str.replace(r'(https?:\/\/www.nytimes.com\/(interactive)\/\d+\/)','', regex=True)
    df[x] = df[x].str.replace(r'(https?:\/\/www.nytimes.com\/(video)\/)','', regex=True)
    df[x] = df[x].str.replace(r'(https?:\/\/www.nytimes.com\/)','', regex=True)
    df[x] = df[x].str.replace(r'(https?:\/\/brandedplaylist.nytimes.com\/)','', regex=True)
    df[x] = df[x].str.replace(r'((us)\/)','', regex=True)
    df[x] = df[x].str.replace(r'(\/.+)','', regex=True)
    df[x] = df[x].str.replace(r'\s+','', regex=True)
    df[x] = df[x].str.replace(r'(.+(.html))','us', regex=True)
    return df[x]

In [8]:
df = pd.read_csv('../data/raw/raw-data.csv')

In [9]:
df

,date,url,headline,articles,doc_type,material_type,section,keywords,label
0,2020-01-01,https://www.nytimes.com/2019/12/31/us/texas-church-shooting-white-settlement.html,‘Battling a Demon’: Drifter Sought Help Before Texas Church Shooting,"The gunman who shot two parishioners at the West Freeway Church of Christ had come earlier looking for food and money, church leaders said.",article,News,NaN,"['Churches (Buildings)', 'Murders, Attempted Murders and Homicides']",us
1,2020-01-01,https://www.nytimes.com/2019/12/31/opinion/for-profit-college-veterans.html,Protect Veterans From Fraud,Congress could do much more to protect Americans who have served their country from predatory for-profit colleges.,article,Editorial,NaN,"['Veterans', 'For-Profit Schools', 'Financial Aid (Education)', 'Frauds and Swindling', 'Colleges and Universities']",opinion
2,2020-01-01,https://www.nytimes.com/2019/12/31/health/e-cigarettes-flavor-ban-trump.html,F.D.A. Plans to Ban Most E-Cigarette Flavors but Menthol,The tobacco and vaping industries and conservative allies intensively lobbied against a ban on popular flavored e-cigarettes.,article,News,NaN,"['E-Cigarettes', 'Recalls and Bans of Products', 'Menthol', 'Nicotine', 'Teenagers and Adolescence', 'Regulation and Deregulation of Industry', 'Smoking and Tobacco', 'Lobbying and Lobbyists', 'United States Politics and Government', 'your-feed-healthcare']",health
3,2020-01-01,https://www.nytimes.com/2019/12/31/crosswords/daily-puzzle-2020-01-01.html,‘It’s Green and Slimy’,Christina Iverson and Jeff Chen ring in the New Year.,article,News,NaN,['Crossword Puzzles'],crosswords
4,2020-01-01,https://www.nytimes.com/2019/12/31/pageoneplus/corrections-jan-1-2020.html,"Corrections: Jan. 1, 2020","Corrections that appeared in print on Wednesday, Jan. 1, 2020.",article,Correction,NaN,[],us
...,...,...,...,...,...,...,...,...,...
4475,2020-01-31,https://www.nytimes.com/2020/01/31/sports/basketball/lakers-blazers-kobe.html,Lakers Fall to Blazers on Emotional Night Honoring Kobe Bryant,It was the Lakers’ first game since Bryant and his daughter Gianna died in a helicopter crash on Sunday. LeBron James gave a heartfelt speech before the game.,article,News,NaN,['Basketball'],sports
4476,2020-01-31,https://www.nytimes.com/2020/01/31/sports/olympics/alberto-salazar-ban.html,Alberto Salazar Is Suspended by SafeSport After Accusations of Verbal Abuse,"The famed running coach was already barred from his sport for four years because of doping violations. He is now temporarily banned by the U.S. Center for SafeSport, an action that could produce a lifetime ban.",article,News,NaN,"['Running', 'Coaches and Managers']",sports
4477,2020-01-31,https://www.nytimes.com/2020/01/31/health/cpr-elderly.html,"CPR, by Default","When very old patients suffer cardiac arrest, doctors usually try to revive them — even if they were already near death.",article,News,NaN,"['Hospitals', 'Defibrillators', 'Living Wills and Health Care Proxies', 'Elderly', 'Doctors', 'Cardiopulmonary Resuscitation (CPR)']",health
4478,2020-01-31,https://www.nytimes.com/video/us/politics/100000006953678/impeachment-highlights.html,Impeachment Trial Highlights: A Showdown Over Calling Witnesses,"Senators rejected a call for additional witnesses in President Trump’s impeachment trial, dealing a fatal blow to efforts by Democrats to bring about new evidence.",multimedia,Video,NaN,"['Impeachment', 'Trump-Ukraine Whistle-blower Complaint and Impeachment Inquiry', 'United States Politics and Government']",politics


In [ ]:
if __name__ == '__main__':
    response = send_request(date)
    df = parse_response(response)
    df['label'] = extract_label('label')
    df.to_csv('../data/raw/raw-data.csv', index=None)